# LIN 373 Machine Learning Toolbox for Text Analysis, Spring 2019

# Homework 2 - due Tuesday Feb 25 2020 at 2:00pm

For this homework you will hand in (upload) to canvas:
- a notebook renamed ``hw2_YourEID.ipynb``
- a PDF version named `hw2_YourEID.pdf`

__Before submitting__, please reset your kernel and rerun everything from the beginning (`Kernel` >> `Restart and Run All`) an ensure your code outputs the correct answer. In addition, you will upload a PDF version of this notebook with all outputs included (`File` >> `Download As` >> `PDF via LaTeX`). If you do not have latex on your machine, you should do this by using print preview and then printing as PDF (`File` >> `Print Preview`)

A perfect solution for this homework is worth **100** points. **There is also a 10 point bonus assignment.** For non-programming tasks, you must show work to get partial credit. For programming tasks, make sure that your code can run using Python 3.5+. If you cannot complete a problem, include as much pseudocode as possible for partial credit. However, make sure it does not have any output errors. **If there are any output errors, half of the points for that problem will be automatically deducted.**

Collaboration: you are free to discuss the homework assignments with other students and work towards solutions together.  However, all of the code you write must be your own! There is a thread on Piazza where you can look for a study group: https://piazza.com/class/k5o768x0bq6ai?cid=10

Review extension and academic dishonesty policy here: https://jjessyli.github.io/courses/lin373#extension-policy

For typing up your answers to problems 1 and 2, information can be found about Markdown cells for Jupyter Notebooks here: https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Working%20With%20Markdown%20Cells.html


### Please list any collaborators here:
- 



# Problem 1: Naive Bayes

We'll do a bit of manual parameter estimation here.

## **(a)** (20 points) 
Calculate the sufficient parameters for Naive Bayes using the data in the figure below, that
is, the prior class probabilities and the conditional probabilities for all of the symbols.

X | Y
--| --
a b b b c b | 1
c a c c c b | -1
c c c b | -1
b a b b b | 1
b c a b b | ???


Group 1:
abbbcb : (a: 1, b: 4, c: 1)
babbb  : (a: 1, b: 4, c: 0)
totals : (a: 2, b: 8, c: 1)
size   : 11

Group -1:
cacccb : (a: 1, b: 1, c: 4)
cccb   : (a: 0, b: 1, c: 3)
totals : (a: 1, b: 2, c: 7)
size   :  10

priors
p(1) = 1/2
p(-1) = 1/2

conditionals
p(a|1) = 2/11
p(b|1) = 8/11
p(c|1) = 1/11

p(a|-1) = 1/10
p(b|-1) = 2/10
p(c|-1) = 7/10




## **(b)** (5 points) 
Using these, calculate the most likely class (1 or -1) for the unlabeled example (currently
labeled `???`).

Group1
p(1)*P(b|1)^3*P(c|1)*P(a|1) = (1/2) * (8/11)^3* (2/11) * (1/11) = 0.00317911717



Group2
P(-1)*P(b|-1)^3*P(a|-1)*P(c|-1) = (1/2) * (2/10)^2 *(1/10)*(7/10) = 0.0014


The unknown is predicted to be 1

# Problem 2: Language Modeling

In this problem, we will implement our own Shakespeare and Austen language models!

### Data

We will be working with a few different files. 
- `shakespeare.txt`

- `shakespeare_short.txt`
- `austen.txt`
- `austen_short.txt`
- `test.txt`

You can use the "short" text files to do testing and debugging as the other files will take a little while to run! 

## (a) Clean text. (5 points) 

Create a function `sent_transform(sent_string)`, such that when given a sentence as a string, it would return a `list` of words, lower-cased. Use NLTK's `word_tokenize` function to tokenize the sentence. For now, you can use any random string to test this function.

```
>>> sent_transform("ROSALIND. Why, whither shall we go?")
>>> ['rosalind', '.', 'why', ',', 'whither', 'shall', 'we', 'go', '?']
```

In [1]:
import nltk


def sent_transform(sentence_string):
    words = nltk.word_tokenize(sentence_string.lower())
    return words

print(sent_transform("ROSALIND. Why, whither shall we go?"))


['rosalind', '.', 'why', ',', 'whither', 'shall', 'we', 'go', '?']


## (b) Find n-grams. (10 points) 

Create a function `make_ngram_tuples(words, n)` that returns a sequence of all n-grams seen in the input, in order.  The function returns a sequence of tuples where each tuple is of the form `(context, word)`.  The context is a tuple of the preceding n−1 words for each word; if the number of preceding words is smaller than n−1, place a `<s>` symbol in place of each missing context.  Close each sentence with an additional token `</s>`.  You can assumen n>1, that is, we are interested in enumerating bigrams, trigrams etc, and not unigrams.

For now, you can use any random string to test this function.

```
>>> samples = [’she’, ’eats’, ’happily’]
>>> make_ngram_tuples(samples, 2)
[((’<s>’,), ’she’), ((’she’,), ’eats’), ((’eats’,), ’happily’), ((’happily’,), ’</s>’)]
>>> make_ngram_tuples(samples, 3)

[((’<s>’, ’<s>’), ’she’), ((’<s>’, ’she’), ’eats’), ((’she’, ’eats’), ’happily’),((’eats’, ’happily’), ’</s>’)]
```

In [343]:
def make_ngram_tuples(samples,n):
    ngram_tuple = []
    for ind in range(len(samples)):
        ngram_tuple.append((helper_fun(ind,n,samples)
                           ,samples[ind]))
    ngram_tuple.append((helper_fun(len(samples),n, samples),'<s>'))
    return ngram_tuple
        
        
def helper_fun(ind, n,samples):
    tuple_context = []
    backwards = ind - n + 1
    while backwards != ind:
        if backwards < 0:
            tuple_context.append('<s>')
        else:
            tuple_context.append(samples[backwards])
        backwards +=1
    return tuple(tuple_context)
        
    
    
print(make_ngram_tuples( ['she', 'eats', 'happily'],3))

[(('<s>', '<s>'), 'she'), (('<s>', 'she'), 'eats'), (('she', 'eats'), 'happily'), (('eats', 'happily'), '<s>')]


## (c)  Building a bigram language model (30 points)

 We are now ready to estimate a bigram language model from the training data.  We will use **Laplace smoothing (add-1)**. With our model on a laptop with 16g RAM, it took <20 seconds to run the Shakespeare model.

Define a class `BigramModel` with the following instance methods:

* `__init__(self, inputfile)`: the  constructor  that  builds  a  bigram  language  model  from  an input file.  Assume  file paths. The language model ought to be able to handle words not seen in the training data.  Such words will most certainly appear if the LM is used in some application to estimate the likelihood of new data.   There  are  many  ways  to  incorporate  unknown  vocabulary  in  a  language  model.   In  this problem, we will take all words that appear only once and replace them with the symbol `<UNK>`. 

* `logprob(self, context, word)`:  returns the log probability of the current word given its context. Note that you need to pre-process the input: words should be lower-cased and out of vocabulary words should be replaced by `<UNK>`s.

* `get_ppl(self, testfile)`: given a testfile file, returns the perplexity of the model computed for that particular text file.

In [345]:
import nltk
import numpy as np
from collections import Counter,defaultdict


class BigramModel(object):
    
    def __init__(self, inputfile, alpha):
        
        # initials
        self.alpha = alpha
        self.clean_input = self.clean_input_func(inputfile)
        self.word_counts = Counter(self.clean_input)
        self.unknown_words_dict = self.getUnknownWords(self.word_counts)
        self.vocab = set(self.unknown_words_dict.keys())
        self.vocab_size = len(self.vocab)
        
        
        # bigram part
        self.bigrams = nltk.bigrams(self.clean_input)
        self.bigram_dict = self.get_bigram_dictionary(self.bigrams)
        
        
    #############################################################################################################33
    # bigram functions
    #
    #
    #################################################################################################################
    def get_bigram_freq(self, word1,word2):
        return np.log2(self.bigram_dict[word1][word2] + self.alpha)-np.log2(self.word_counts[word2] + (self.alpha * self.vocab_size))
    
        # I am assuming that the context is given as a string and the word is given as a string
    def logprob(self, context, word): 
        sent = nltk.bigrams(nltk.word_tokenize(context.lower()) + [word])
        return np.sum([(self.get_bigram_freq(self.get_valid_word(word1), self.get_valid_word(word2))
                       ) for word1, word2 in sent])
    
    def logprob_sentences(self, sent): 
        return np.sum([(self.get_bigram_freq(self.get_valid_word(word1), self.get_valid_word(word2))
                       ) for word1, word2 in sent])
    
    def get_ppl(self, testfile):
        sentences = self.clean_input_func(testfile)
        sent = [bi for bi in nltk.bigrams(sentences)]
        log_sent_freq = self.logprob_sentences(sent)
        return 2 ** (-1 * log_sent_freq/len(sent))
    

    
  

    ################################################################################################################
    # preprocessing and helper functions for txt files
    #
    #
    ##################################################################################################################


    # function gets amount of words that have a count of 1
    # Makes new dictionary that has all words with more than one count and the count of all unknown words
    def getUnknownWords(self,word_dict):
        unknown_words = 0
        dictionary_with_unknown_words = {}
        for k,v in word_dict.items():
            if v == 1:
                unknown_words += 1
            else:
                dictionary_with_unknown_words[k] = v
        dictionary_with_unknown_words["^unk^"] = unknown_words
        return dictionary_with_unknown_words
    
    
    #
    # This function creates a bigram dictionary. It gets counts for all bigrams
    def get_bigram_dictionary(self,bigrams):
        d_bigram = defaultdict(Counter)
        for i in bigrams:
            d_bigram[self.get_valid_word(i[0])][self.get_valid_word(i[1])] +=1
        return d_bigram
    
    #
    # This function checks if a word is in the vocab and returns the word or unk if not in vocab
    def get_valid_word(self,word):
        if word.lower() in self.vocab:
            return word.lower()
        else:
            return "^unk^"
        
    #
    # This function will open the text file,
    # separate sentences out using nltk
    # at an ending marking and starting marker
    # Then concat a word tokenized list of sentences together
    # we get a bag of words where sentences are distinguised by the markers
    def clean_input_func(self,input_file):
        text = []
        with open(input_file, encoding="utf8") as file:
             sentences = nltk.sent_tokenize(file.read())
        for sent in sentences:
            sent = sent.lower()
            whole_sentence = ['<s>'] + nltk.word_tokenize(sent) + ['</s>']
            text = text + whole_sentence
        return text



In [ ]:
my_bigram_Shake = BigramModel("data/shakespeare.txt",.01)
print(my_bigram_Shake.bigram_dict['the'].most_common(5)[1])

In [346]:
my_bigram_Shake = BigramModel("data/shakespeare.txt",.01)
my_bigram_Austen = BigramModel("data/austen.txt",.01)
print(my_bigram_Austen.get_ppl("data/test.txt"))
print(my_bigram_Shake.get_ppl("data/test.txt"))



115.46666854333515
37.06193930567681


## (d) Random text generator

(15 points) Now, we are ready to generate some Shakespeare text! Starting with the sentence start symbol `<s>`, at each step, use the previously generated word as context, and sample one of the top 5 words that follow this word. We stop whenever we hit `</s>`, or when we have generated a 20-word sentence, whichever is earlier.

Implement a function `text_generator(bigramlm, n)` that takes a BigramModel---trained on our Shakespeare text---as input, and generates `n` random sentences.

In [356]:
import random


def text_generator(bigramLm, n):
    for i in range(n):
        start = ['<s>']
        end = '</s>'
        next_word = start
        while next_word[-1] != end and len(next_word) < 21:
            top_5 = bigramLm.bigram_dict[next_word[-1]].most_common(5)
            rand_range_fix = random_fix(top_5)
            random_num = random.randrange(0,rand_range_fix,1)
            future_word = top_5[random_num][0]
            if future_word == "^unk^":
                continue
            next_word += [future_word]
        
        # put words into sentence and print
        print(" ".join(next_word))
    return "done"
    
# sometimes there isn't a top 5 because there are too few words
# so we need this function
def random_fix(top5):
    top_5_length = len(top5)
    if top_5_length < 5:
        return top_5_length
    else:
        return 5
    
    
    
    
print(text_generator(my_bigram_Austen, 20))
    

<s> she had not be a very good humour and the house , and i am glad of her own room
<s> it , and i have the world to have not know what he could have a few days together ;
<s> `` oh no one . ' </s>
<s> it was in their acquaintance . ' '' was the world to the same . </s>
<s> `` yes ; `` you , and she felt . </s>
<s> it , she could do . '' cried mary , and i have no , '' said fanny . </s>
<s> it would have the house in a great deal to her to see her to the same moment ; i
<s> she had a great deal to her . ' said he was the same time , `` but it was
<s> it , `` but i am very well to see you are , she could not have been so very
<s> she would have the world ! ' '' cried , i am glad to her . ' '' said ,
<s> the same , and the first , '' cried mrs. allen was not know that the same . '' was
<s> i am sure , '' was a great , '' cried catherine 's being in their father had not to
<s> she felt . </s>
<s> she would be the house , and , she would have no doubt , '' was to her ; but
<s> it was not h

## (e) Authorship attribution (15 points)

If we have built language models of multiple authors, they can be used to check the author of an unknown piece of writing. Concretely, for the unknown text, we can run the file through each known author's language model, and use perplexity to predict which author the unknown text is most likely to belong to.

In this problem, we build four language models:
- a  Shakespeare language model
- a  Jane Austen language model

Once you have trained both models, use the `get_ppl` function from each language model on the file `test.txt`. 

Who wrote the text?

In [351]:
my_bigram_Shake = BigramModel("data/shakespeare.txt",.01)
my_bigram_Austen = BigramModel("data/austen.txt",.01)
print(my_bigram_Austen.get_ppl("data/test.txt"))
print(my_bigram_Shake.get_ppl("data/test.txt"))

115.46666854333515
37.06193930567681


Jane Austen wrote the test script


## Bonus (10 points): Data Annotation

This bonus problem is about a first-hand experience of **gathering** data. We will first annotate data for an existing research project, and later in the semester, we will analyze this dataset that y'all annotated. We hope you find the data fun to look at!

If you choose to do this problem, we also really, really appreciate your help in this research project that's ongoing in the Linguistics department. The annotations will *not* be anonymous because of grading reasons (we need to know that you've done the task to award you points), but *no* personal information will be collected.

We will be using a popular crowdsourcing platform: the Amazon Mechanical Turk. Here, we will look at it from the *annotator*'s angle; in class later in the semester, we will also look at it from a *data collector*'s angle. Since this is a small class experiment, we will only work with their sandbox version, i.e., unfortunately, no real money involved.

* 1) In order to annotate data, you need to setup yourselves as an Amazon Mechanical Turk (AMT) "worker"
  * a) go to this url and log in with your amazon.com credentials (use the same user/password when you shop on Amazon): https://workersandbox.mturk.com/
  * b) complete the steps to register as an AMT worker
  
* 2) Find the AMT task:
  * a) after registering to be a worker (see above), go to this url: https://workersandbox.mturk.com/projects/3QZMXSMFJ7GIP9G367LB3IAC25JKL2/tasks
  * b) you should see a page like the one below. Confirm the requester is "CompDisc” and the title begins with “What is the intent...”
  <img src="images/1.png" alt="bonus question" width="600"/>
  
* 3) Complete the AMT task:
  * a) Click on the orange “Accept" button in the upper-right corner
  <img src="images/2.png" alt="bonus question" width="600"/>
  * b) Read and follow the instructions. After answering all the questions, click on the “Submit” button at the bottom-left of the page.
  <img src="images/3.png" alt="bonus question" width="600"/>
  
* c) **(Optional)** If you find this task fun, repeat the above steps to complete more HITs (you can complete up to 6 HITs). The instructions are the same so you can skip them by clicking on “Click to hide/show”.
  <img src="images/4.png" alt="bonus question" width="600"/>
  
* d) Copy your worker id in the upper left corner of the page, and paste it below:

In [ ]:
AR6AHNQNG8HN2

Since we would like to bring this experiment to the public, please share any feedback, good or bad! For example, let us know if you felt there were any unclear or ambiguous parts, or anything that could be clarified or made more intuitive to relieve the need for lengthy instructions.

Enter any comments below: